In [ ]:
!pip install --upgrade pip
!pip install pandas numpy scipy implicit scikit-learn surprise matplotlib networkx


In [ ]:
import os
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)



## Import du dataset

In [ ]:
DATA_DIR = "data_final_project/KuaiRec 2.0/data"

df_inter = pd.read_csv(os.path.join(DATA_DIR, "big_matrix.csv"))

df_cat = pd.read_csv(os.path.join(DATA_DIR, "item_categories.csv"))
df_daily = pd.read_csv(os.path.join(DATA_DIR, "item_daily_features.csv"))
df_user = pd.read_csv(os.path.join(DATA_DIR, "user_features.csv"))
df_soc = pd.read_csv(os.path.join(DATA_DIR, "social_network.csv"))


print("Interactions :", df_inter.shape)
print("Catégories :",   df_cat.shape)
print("Daily feat :",   df_daily.shape)
print("Social net :",   df_soc.shape)
print("User feat :",    df_user.shape)

In [ ]:
display(df_inter.head())
display(df_cat.head())
display(df_daily.head())
display(df_soc.head())
display(df_user.head())

### Schéma des tables et relations

Nous disposons de 5 DataFrames principaux :

1. **`df_inter`** – historique des interactions utilisateur ↔ vidéo  
   - Clé primaire implicite : couple `(user_id, video_id, timestamp)`  
   - Champs clés :  
     - `user_id` : identifiant de l’utilisateur  
     - `video_id` : identifiant de la vidéo  
     - `play_duration`, `video_duration`  
     - `watch_ratio` = `play_duration / video_duration`  

2. **`df_cat`** – catégories / attributs par vidéo  
   - Clé primaire : `video_id`  
   - `feat` : liste de catégories (p. ex. `[27, 9]`)  

3. **`df_daily`** – statistiques journalières par vidéo  
   - Clé unique : `(video_id, date)`  
   - Ex. `play_cnt`, `like_cnt`, `share_cnt`, etc.  

4. **`df_soc`** – relations sociales de chaque utilisateur  
   - Clé primaire : `user_id`  
   - `friend_list` : liste des IDs d’amis  

5. **`df_user`** – profil et comportements des utilisateurs  
   - Clé primaire : `user_id`  
   - Champs descriptifs : `user_active_degree`, `follow_user_num`, `register_days`, plus one-hot features…  

Les principales relations sont :  
- `df_inter.user_id` → `df_user.user_id` (1:N)  
- `df_inter.video_id` → `df_cat.video_id` (N:1)  
- `df_inter.video_id` → `df_daily.video_id` (N:N via date)  
- `df_user.user_id` → `df_soc.user_id` (1:1)  


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt


G = nx.DiGraph()

tables = ["df_user", "df_soc", "df_inter", "df_cat", "df_daily"]
G.add_nodes_from(tables)

edges = [
    ("df_inter", "df_user"),   
    ("df_inter", "df_cat"),    
    ("df_inter", "df_daily"),  
    ("df_user", "df_soc"),     
]
G.add_edges_from(edges)

pos = {
    "df_user":    (0, 1),
    "df_soc":     (0, 0),
    "df_inter":   (1, 0.5),
    "df_cat":     (2, 1),
    "df_daily":   (2, 0),
}

plt.figure(figsize=(8, 5))
nx.draw(G, pos, with_labels=True, node_size=2500, node_color="lightblue", arrowsize=20)
plt.title("Diagramme des tables et relations")
plt.axis("off")
plt.show()


In [ ]:
df_daily['upload_type'] = df_daily['upload_type'].fillna('unknown')

df_soc['friend_list'] = df_soc['friend_list'].fillna('').apply(lambda x: x.split(',') if x else [])

num_cols = df_user.select_dtypes(include='number').columns
for col in num_cols:
    if df_user[col].isna().any():
        median = df_user[col].median()
        df_user[col].fillna(median, inplace=True)

print("Valeurs manquantes restantes :")
for name, df in [('daily', df_daily), ('social', df_soc), ('user', df_user)]:
    print(name, df.isna().sum().sum())



1. **`upload_type` → `'unknown'`**  
   On remplace les valeurs manquantes par une catégorie explicite, pour éviter les erreurs lors des encodages ultérieurs.

2. **`friend_list` → liste d’IDs**  
   - On transforme d’abord les `NaN` en chaîne vide.  
   - Puis on découpe chaque chaîne sur la virgule pour obtenir une liste d’identifiants, ou une liste vide si aucun ami.

3. **Imputation médiane sur `df_user`**  
   - On sélectionne toutes les colonnes numériques.  
   - Pour chaque colonne ayant des `NaN`, on calcule sa médiane et on remplace les `NaN` par cette valeur, afin de conserver la distribution centrale sans être influencé par les outliers.

4. **Vérification**  
   - On totalise les `NaN` restants pour s’assurer qu’il n’en subsiste plus dans les tables traitées.


In [ ]:
df_inter = df_inter.merge(
    df_cat.rename(columns={'feat': 'video_category'}),
    how='left', on='video_id'
)

print("Après fusion catégorie :", df_inter.shape)
print(df_inter[['video_id', 'video_category']].head())



1. **Renommage de colonne**  
   On commence par renommer `feat` en `video_category` dans `df_cat` pour clarifier son rôle et éviter les ambiguïtés une fois fusionné.

2. **Fusion (merge)**  
   - `how='left'` : on réalise une jointure à gauche, c’est-à-dire que chaque ligne de `df_inter` est conservée, même si la vidéo n’a pas de catégorie associée dans `df_cat`.  
   - `on='video_id'` : on s’appuie sur l’identifiant de la vidéo pour rattacher les catégories.

3. **Vérification**  
   - `df_inter.shape` affiche le nombre de lignes et de colonnes après la fusion (doit correspondre au nombre initial de lignes de `df_inter`, avec une colonne supplémentaire).  
   - L’aperçu (`.head()`) montre que, pour chaque `video_id`, on dispose désormais de la liste `video_category` issue de `df_cat`.


## Feature Engineering

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse as sp

In [ ]:
df_inter['timestamp'] = pd.to_datetime(df_inter['timestamp'], unit='s')
df_inter['date']      = df_inter['timestamp'].dt.date
df_inter['hour']      = df_inter['timestamp'].dt.hour
df_inter['dayofweek'] = df_inter['timestamp'].dt.dayofweek 

df_inter[['timestamp','date','hour','dayofweek']].head()



1. **`pd.to_datetime(..., unit='s')`**  
   On convertit la colonne `timestamp` en objets `datetime64[ns]` de pandas, ce qui permet ensuite d’accéder à toutes ses composantes temporelles

2. **`.dt.date`**  
   L’attribut `dt.date` retourne uniquement la partie date (type `datetime.date`), sans information d’heure ni de fuseau

3. **`.dt.hour`**  
   L’attribut `dt.hour` extrait l’heure (entier de 0 à 23) de chaque timestamp, utile pour analyser les pics d’activité selon l’heure de la journée

4. **`.dt.dayofweek`**  
   L’attribut `dt.dayofweek` renvoie un entier de 0 (lundi) à 6 (dimanche), permettant d’étudier les variations journalières (weekend vs jours de semaine)

Cette transformation enrichit `df_inter` de quatre nouvelles colonnes temporelles clés, ouvrant la voie à des analyses temporelles fines (saisonnalité horaire, comportements selon le jour de la semaine, etc.).


In [ ]:
user_agg = df_inter.groupby('user_id').agg(
    user_total_views      = ('video_id','count'),
    user_unique_videos    = ('video_id','nunique'),
    user_mean_watch_ratio = ('watch_ratio','mean'),
    user_std_watch_ratio  = ('watch_ratio','std')
).reset_index()

df_user = df_user.merge(user_agg, how='left', on='user_id')
df_user[user_agg.columns] = df_user[user_agg.columns].fillna(0)



1. **GroupBy + agg**  
   - On regroupe `df_inter` par `user_id`.  
   - Pour chaque utilisateur, on calcule quatre métriques :  
     - **`user_total_views`** : nombre total d’enregistrements (vues).  
     - **`user_unique_videos`** : nombre de vidéos distinctes visionnées.  
     - **`user_mean_watch_ratio`** : ratio moyen de visionnage (`play_duration`/`video_duration`).  
     - **`user_std_watch_ratio`** : dispersion de ces ratios (indicateur de régularité).

2. **Fusion (`merge`)**  
   - Avec `how='left'`, on ajoute ces nouvelles colonnes à `df_user`.  
   - Les utilisateurs sans historique dans `df_inter` recevront des `NaN`.

3. **Imputation des NaN par 0**  
   - Pour tous les utilisateurs n’ayant pas d’interactions, on définit ces métriques à 0 (pas de vues, pas de diversité, etc.).

4. **Résultat**  
   - `df_user` est enrichi de quatre nouvelles fonctionnalités quantitatives

In [ ]:
item_agg = df_inter.groupby('video_id').agg(
    item_view_count       = ('user_id','count'),
    item_unique_viewers   = ('user_id','nunique'),
    item_mean_watch_ratio = ('watch_ratio','mean')
).reset_index()

df_cat = df_cat.merge(item_agg, how='left', on='video_id')
df_cat[item_agg.columns] = df_cat[item_agg.columns].fillna(0)


1. **GroupBy + agg**  
   - On regroupe `df_inter` par `video_id`.  
   - Pour chaque vidéo, on calcule trois métriques :  
     - **`item_view_count`** : nombre total de vues (tous utilisateurs confondus).  
     - **`item_unique_viewers`** : nombre d’utilisateurs distincts ayant visionné la vidéo.  
     - **`item_mean_watch_ratio`** : ratio moyen de visionnage, indicateur de la rétention moyenne.

2. **Fusion (`merge`)**  
   - Avec `how='left'`, on ajoute ces colonnes à `df_cat` tout en conservant toutes les vidéos existantes.  
   - Si une vidéo n’apparaît pas dans `df_inter`, elle recevra un `NaN` pour ces métriques.

3. **Imputation des NaN par 0**  
   - Pour les vidéos sans aucune interaction, on fixe ces métriques à 0 (aucune vue, aucun spectateur, ratio de visionnage nul).

4. **Résultat**  
   - `df_cat` est désormais enrichi de trois nouvelles fonctionnalités descriptives, prêtes à être utilisées pour l’entraînement ou l’analyse.

In [ ]:
daily_plays = df_inter.groupby(['video_id','date']).size().reset_index(name='plays')
daily_plays['date'] = pd.to_datetime(daily_plays['date'])

daily_plays = daily_plays.sort_values(['video_id','date'])
daily_plays['plays_7d'] = daily_plays.groupby('video_id')['plays']\
                                     .transform(lambda x: x.rolling(7, min_periods=1).sum())

df_daily['date'] = pd.to_datetime(df_daily['date'])
daily_plays['date'] = pd.to_datetime(daily_plays['date'])

df_daily = df_daily.merge(
    daily_plays[['video_id','date','plays_7d']],
    how='left',
    on=['video_id','date']
)

df_daily['plays_7d'].fillna(0, inplace=True)

1. **Agrégation journalière (`groupby` + `size`)**  
   On commence par compter, pour chaque couple `(video_id, date)`, le nombre d’enregistrements dans `df_inter`. Ce comptage représente le nombre total de lectures de la vidéo ce jour-là.

2. **Conversion en `datetime`**  
   Pour pouvoir trier et appliquer une fenêtre glissante sur une colonne de dates, il faut travailler avec le type `datetime64[ns]`.

3. **Tri chronologique**  
   Le tri par `(video_id, date)` garantit que, lors de l’application de la fenêtre glissante, les lectures sont correctement ordonnées du plus ancien au plus récent.

4. **Fenêtre glissante de 7 observations**  
   - `groupby('video_id')['plays']` : on cible la série des lectures pour chaque vidéo séparément.  
   - `.transform(lambda x: x.rolling(7, min_periods=1).sum())` :  
     - La méthode `rolling(window, min_periods)` crée une **fenêtre glissante** de largeur fixe (ici 7 lignes).  
     - `min_periods=1` permet d’obtenir une valeur même si moins de 7 jours sont disponibles (utile pour les premières dates).  
     - La somme de la fenêtre donne, pour chaque date, le total des lectures des 7 derniers jours.

5. **Fusion des données**  
   On rattache la colonne `plays_7d` à `df_daily` via une jointure gauche (`how='left'`) sur `video_id` et `date`. Cela ajoute la métrique à chaque ligne de `df_daily`.

6. **Imputation finale**  
   Pour les vidéos ou dates n’apparaissant pas dans `daily_plays` (par exemple, moins d’une semaine de données), `plays_7d` sera `NaN` après la fusion : on remplace ces valeurs par 0, signifiant « aucune lecture sur la période ».


In [ ]:
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
cat_ohe = ohe.fit_transform(df_cat[['feat']])
cat_ohe = pd.DataFrame(cat_ohe, columns=[f"cat_{c}" for c in ohe.categories_[0]])
df_cat = pd.concat([df_cat.reset_index(drop=True), cat_ohe], axis=1)

vect = CountVectorizer(token_pattern=r"(?u)\b\w+\b", min_df=10)
tag_sparse = vect.fit_transform(df_daily['video_tag_name'].fillna(''))

tags_df = pd.DataFrame.sparse.from_spmatrix(tag_sparse, 
             index=df_daily.index, columns=[f"tag_{t}" for t in vect.get_feature_names_out()])
df_daily = pd.concat([df_daily, tags_df], axis=1)


- **OneHotEncoder** : on instancie `OneHotEncoder` de scikit‑learn avec `sparse_output=False` pour obtenir un tableau dense (au lieu d’une matrice creuse).  
- **fit_transform(df_cat[['feat']])** : la méthode `fit_transform` apprend sur la colonne `feat` (liste d’identifiants de catégories) et renvoie une matrice de 0/1 indiquant la présence de chaque catégorie pour chaque vidéo.  
- **Construction de `cat_ohe`** : on convertit la matrice numpy en DataFrame pandas, en nommant les colonnes comme `cat_{c}` pour chaque catégorie `c` identifiée par l’encodeur.  
- **Concaténation** : `pd.concat([...], axis=1)` fusionne horizontalement `df_cat` et `cat_ohe`, ajoutant autant de nouvelles colonnes binaires que de catégories uniques.  

- **CountVectorizer** : instancié avec `token_pattern=r"(?u)\b\w+\b"` pour capturer chaque mot (même d’un seul caractère) et `min_df=10` pour ne conserver que les tokens présents dans au moins 10 documents, filtrant ainsi le bruit.  
- **fillna('')** : on remplace les `NaN` de `video_tag_name` par chaîne vide afin que `CountVectorizer` ne plante pas sur valeurs manquantes.  
- **fit_transform** : génère un objet `scipy.sparse.csr_matrix` de comptages de tokens par ligne (une ligne = une vidéo/jour).  
- **from_spmatrix** : `pd.DataFrame.sparse.from_spmatrix` convertit la matrice creuse en DataFrame pandas à colonnes creuses (`SparseArray`), économisant de la mémoire quand la majorité des compteurs est nulle.  
- **naming** : chaque colonne de tags est préfixée par `tag_{t}` pour chaque token `t` du vocabulaire appris, facilitant l’identification dans le DataFrame.  
- **Concaténation finale** : on rassemble `df_daily` et `tags_df` pour enrichir les données journalières de features textuelles exploitables en apprentissage.

In [ ]:

user_ids = df_inter['user_id'].unique()
item_ids = df_inter['video_id'].unique()
uid2idx = {u:i for i,u in enumerate(user_ids)}
iid2idx = {i:j for j,i in enumerate(item_ids)}

rows = df_inter['user_id'].map(uid2idx)
cols = df_inter['video_id'].map(iid2idx)
data = df_inter['watch_ratio'].values

R = sp.csr_matrix((data, (rows, cols)), shape=(len(user_ids), len(item_ids)))
print("Sparse matrix R shape:", R.shape)


1. **Extraction des identifiants uniques**  
   - `user_ids` et `item_ids` contiennent respectivement tous les `user_id` et `video_id` distincts présents dans `df_inter`.  
   - Cela fixe les dimensions de la matrice (nombre d’utilisateurs × nombre de vidéos).

2. **Création des dictionnaires de mapping**  
   - `uid2idx` associe chaque `user_id` à un index de 0 à `n_users − 1`.  
   - `iid2idx` fait de même pour chaque `video_id`, de 0 à `n_items − 1`.  
   - Ces mappings servent à positionner correctement chaque interaction dans la matrice.

3. **Génération des vecteurs `rows` et `cols`**  
   - On transforme la colonne `user_id` en indices de lignes avec `.map(uid2idx)`,  
   - et la colonne `video_id` en indices de colonnes avec `.map(iid2idx)`.

4. **Récupération des valeurs**  
   - `data` contient la liste des `watch_ratio` correspondant à chaque interaction (i.e., chaque ligne de `df_inter`).

5. **Construction de la CSR (Compressed Sparse Row)**  
   - `sp.csr_matrix((data, (rows, cols)), shape=(n_users, n_items))` crée une matrice creuse où  
     - `data[i]` est placé en position `(rows[i], cols[i])`.  
   - CSR est efficace en mémoire pour stocker des matrices majoritairement nulles (ici, la plupart des "utilisateur‑vidéo" n’ont pas d’interaction).

La matrice `R` peut désormais être utilisée en entrée pour des algorithmes de factorisation, comme la SVD ou l’ALS.

## Model developpement

In [ ]:
import implicit                       
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
import numpy as np


In [ ]:

idx = np.arange(df_inter.shape[0])
train_idx, test_idx = train_test_split(idx, test_size=0.2, random_state=42)

df_train = df_inter.iloc[train_idx].reset_index(drop=True)
df_test  = df_inter.iloc[test_idx].reset_index(drop=True)

def build_sparse(df):
    rows = df['user_id'].map(uid2idx)
    cols = df['video_id'].map(iid2idx)
    data = df['watch_ratio'].values
    return csr_matrix((data, (rows, cols)), shape=R.shape)

R_train = build_sparse(df_train)
R_test  = build_sparse(df_test)

print("Train matrix shape:", R_train.shape)
print("Test matrix shape :", R_test.shape)


1. **Séparation train/test**  
   - On génère un index de toutes les lignes (`idx`).  
   - `train_test_split(..., test_size=0.2)` choisit 20 % des indices pour le test, 80 % pour l’entraînement, avec une graine (`random_state=42`) pour reproductibilité.

2. **Découpage des DataFrames**  
   - `df_inter.iloc[train_idx]` et `df_inter.iloc[test_idx]` isolent les interactions de chaque sous-ensemble.

3. **Fonction `build_sparse`**  
   - Reprend la même logique que pour `R` : conversion des `user_id` et `video_id` en indices puis assemblage d’une matrice CSR, avec la forme `(n_users, n_items)` identique à `R`.

4. **Matrices `R_train` et `R_test`**  
   - `R_train` contient les ratios de visionnage pour les interactions d’entraînement.  
   - `R_test` contient ceux du test, prêts pour évaluer la qualité du modèle.

5. **Validation**  
   - L’affichage des shapes confirme que la taille des matrices reste `(nombre_utilisateurs, nombre_vidéos)`, mais les données diffèrent selon train/test.

### Modele ALS

In [ ]:
import implicit

model_als = implicit.als.AlternatingLeastSquares(
    factors=50,
    regularization=0.1,
    iterations=20,
    use_gpu=False
)


model_als.fit(R_train)


user    = df_train['user_id'].unique()[0]
uidx    = uid2idx[user]

user_items = R_train[uidx, :]



1. **AlternatingLeastSquares**  
   - Ce modèle factorise la matrice creuse en deux matrices latentes, une pour les items et une pour les users, en résolvant alternativement un système de moindres carrés pour l’un puis l’autre.  
   - Les **`factors`** définissent la dimensionnalité des vecteurs latents,  
   - **`regularization`** pénalise les poids importants pour mieux généraliser,  
   - **`iterations`** contrôle la convergence par itération.

2. **Entraînement (`fit`)**  
   - On fournit `R_train`, où chaque ligne correspond à un utilisateur et chaque colonne à une vidéo, avec pour valeur le `watch_ratio`.  
   - Le calcul peut exiger une matrice transposée (`item × user`), mais la bibliothèque `implicit` gère cela en interne.

3. **Choix d’un utilisateur**  
   - Pour illustrer, on récupère le premier `user_id` du jeu d’entraînement, puis on le mapppe à son index `uidx`.  
   - Cela permettra de générer des recommandations spécifiques à cet utilisateur.

4. **Extraction des interactions existantes**  
   - `user_items = R_train[uidx, :]` renvoie un vecteur creux où chaque entrée non nulle indique un `watch_ratio` pour une vidéo déjà vue.  
   - On peut utiliser ce vecteur pour masquer ces items lors de la génération de recommandations ou pour analyser le comportement passé de l’utilisateur.

> À la suite de cette préparation, on pourra appeler `model_als.recommend(uidx, user_items, N=10)` pour obtenir les 10 meilleures recommandations pour cet utilisateur, en excluant les vidéos déjà vues.

 ### Modele SVD

In [ ]:
from sklearn.decomposition import TruncatedSVD
import numpy as np

n_factors = 20


svd = TruncatedSVD(n_components=n_factors, random_state=42)

U = svd.fit_transform(R_train)      
Sigma = svd.singular_values_        
Vt = svd.components_                




1. **`n_components=20`**  
   On choisit de projeter notre matrice utilisateur–vidéo dans un espace latent de dimension 20. Ce paramètre représente la complexité du modèle :  
   - Trop petit → perte d’information.  
   - Trop grand → risque de surapprentissage.

2. **`TruncatedSVD`**  
   - Semblable à la SVD classique, mais optimisé pour les matrices creuses et de grande dimension.  
   - Ne centre pas les données ; si nécessaire, il faut d’abord **centrer** la matrice (soustraction de la moyenne par utilisateur ou par item).

3. **`fit_transform(R_train)` → `U`**  
   - Calcule les vecteurs propres associés aux 20 premiers vecteurs singuliers.  
   - Renvoie la matrice **U** de dimension `(n_users, 20)` contenant l’“embedding” de chaque utilisateur.

4. **`singular_values_` → `Sigma`**  
   - Tableau des 20 valeurs singulières décroissantes, illustrant l’importance de chaque facteur latent.

5. **`components_` → `Vt`**  
   - Matrice de dimension `(20, n_items)` où chaque colonne est l’“embedding” d’une vidéo dans l’espace latent.

> **Utilisation ultérieure** :  
> Pour prédire un score pour l’utilisateur *u* et la vidéo *i*, on calcule le produit scalaire
> ce qui permet de générer des recommandations en triant ces scores.

## Prediction

In [ ]:
def recommend_svd(user_id, U, Vt, uid2idx, item_ids, N=10):
 
    uidx = uid2idx[user_id]
    user_vec = U[uidx]               
    scores = user_vec.dot(Vt)         
    top_idx = np.argsort(-scores)[:N]
    return [int(item_ids[i]) for i in top_idx]

user = df_train['user_id'].unique()[0]
top10_svd = recommend_svd(user, U, Vt, uid2idx, item_ids, N=10)
print(f"Top 10 SVD pour user {user} : {top10_svd}")



1. **Mapping utilisateur → index**  
   La fonction utilise `uid2idx` pour retrouver la ligne de la matrice **U** correspondant à `user_id`.

2. **Embeddings utilisateur**  
   `user_vec` est un vecteur de dimension `n_factors` représentant les préférences latentes de l’utilisateur.

3. **Calcul des scores**  
   Pour chaque vidéo, on calcule le produit scalaire entre `user_vec` et la colonne correspondante de **Vt**.  
   \[
     \text{score}_{u,i} = U_{u,:} \times Vt_{:,i}
   \]

4. **Tri et sélection**  
   - `np.argsort(-scores)` trie les scores par ordre décroissant.  
   - On prend les `N` premières positions pour obtenir les indices des meilleures vidéos.

5. **Récupération des IDs**  
   Enfin, on convertit ces indices en `video_id` originaux via la liste `item_ids` et on renvoie la liste finale.


## Evaluation

In [ ]:
from scipy.sparse import csr_matrix

def evaluate_als(model, R_train, df_test, uid2idx, item_ids, K=10):
    """
    Évalue un modèle ALS entraîné sur R_train (user×item).
    - model      : instance de AlternatingLeastSquares déjà fit()
    - R_train    : csr_matrix (n_users, n_items)
    - df_test    : DataFrame test avec colonnes ['user_id','video_id']
    - uid2idx    : dict mapping user_id → index
    - item_ids   : array-like index → video_id
    - K          : cutoff pour Precision@K, Recall@K
    """
    n_users, n_items = R_train.shape

    rows = df_test['user_id'].map(uid2idx)
    cols = df_test['video_id'].map({v:i for i,v in enumerate(item_ids)})
    data = np.ones(len(df_test), dtype=np.int8)
    R_test = csr_matrix((data, (rows, cols)), shape=(n_users, n_items))

    precisions = []
    recalls = []

    R_train = R_train.tocsr()
    R_test  = R_test.tocsr()


    for user_id in df_test['user_id'].unique():
        uidx = uid2idx.get(user_id)
        if uidx is None:
            continue

        train_vec = R_train[uidx, :]
        true_vec = R_test[uidx, :]
        true_items = set(true_vec.indices)
        if not true_items:
            continue

        recs_idx, _ = model.recommend(
            userid=uidx,
            user_items=train_vec,
            N=K,
            filter_already_liked_items=True
        )
        rec_items = set(recs_idx)

        hits = rec_items & true_items
        recall    = len(hits) / len(true_items)
        precision = len(hits) / K

        recalls.append(recall)
        precisions.append(precision)

    return {
        'Recall@K': np.mean(recalls),
        'Precision@K': np.mean(precisions)
    }


1. **Construction de `R_test`**  
   On crée, à partir de `df_test`, une matrice creuse binaire de même forme que `R_train` où chaque entrée vaut 1 si l’utilisateur a réellement interagi avec la vidéo dans le jeu de test.

2. **Itération par utilisateur**  
   Pour chaque `user_id` du test, on récupère :
   - `train_vec` : le vecteur des interactions d’entraînement (pour masquer les items déjà vus),
   - `true_vec` : le vecteur binaire des interactions test (pour calculer le rappel).

3. **Recommandation ALS**  
   Avec `model.recommend`, on demande les `K` vidéos les mieux prédites tout en filtrant celles déjà vues en entraînement.

4. **Calcul des métriques**  
   - **Hits** : intersection entre recommandations et véritables items test.  
   - **Recall@K** : proportion des items test retrouvés dans les recommandations.  
   - **Precision@K** : proportion des recommandations qui étaient réellement pertinentes.

5. **Agrégation finale**  
   On renvoie la moyenne de la précision et du rappel sur tous les utilisateurs testés, ce qui donne une évaluation globale du modèle ALS.



In [ ]:
results = evaluate_als(
    model     = model_als,
    R_train   = R_train,
    df_test   = df_test[['user_id','video_id']],
    uid2idx   = uid2idx,
    item_ids  = item_ids,
    K         = 10
)

print(f"ALS - Recall@10    : {results['Recall@K']:.4f}")
print(f"ALS - Precision@10 : {results['Precision@K']:.4f}")


In [ ]:
def evaluate_svd(U, Vt, uid2idx, item_ids, df_test, k=10):
   
    from collections import defaultdict

    itemid2idx = {v: i for i, v in enumerate(item_ids)}

    user_to_items_test = df_test.groupby('user_id')['video_id'].apply(set).to_dict()

    precisions = []
    recalls = []

    for user_id in user_to_items_test:
        if user_id not in uid2idx:
            continue  

        try:
            top_k_pred = recommend_svd(user_id, U, Vt, uid2idx, item_ids, N=k)
        except KeyError:
            continue  

        true_items = user_to_items_test[user_id]

        tp = len(set(top_k_pred) & true_items)

        precision = tp / k
        recall = tp / len(true_items) if true_items else 0

        precisions.append(precision)
        recalls.append(recall)

    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)

    print(f"Average Precision@{k} for SVD: {avg_precision:.4f}")
    print(f"Average Recall@{k} for SVD: {avg_recall:.4f}")


evaluate_svd(U, Vt, uid2idx, item_ids, df_test, k=10)


1. **Mapping inverse des items**  
   On crée `itemid2idx` pour convertir chaque `video_id` en son index, mais ici il n’est finalement pas utilisé directement dans le calcul — l’essentiel est de pouvoir extraire les vrais items test du DataFrame.

2. **Rassemblement des véritables items (`user_to_items_test`)**  
   Grâce à `groupby` + `apply(set)`, on obtient pour chaque `user_id` un ensemble (`set`) des vidéos réellement visionnées dans le jeu de test.

3. **Parcours des utilisateurs**  
   On itère sur chaque utilisateur présent dans `user_to_items_test`. Si l’utilisateur n’existe pas dans le mapping `uid2idx`, on l’ignore (cas cold‑start).

4. **Génération des prédictions SVD**  
   On appelle `recommend_svd`, qui retourne les `k` vidéos les mieux notées pour cet utilisateur, selon le produit scalaire entre son embedding (`U`) et les embeddings des vidéos (`Vt`).

5. **Calcul des vraies positives (`tp`)**  
   On mesure l’intersection entre les prédictions (`top_k_pred`) et l’ensemble des vidéos test (`true_items`).

6. **Précision et rappel**  
   - **Precision@K** = `tp / k`  
   - **Recall@K** = `tp / |true_items|` (avec garde-fou `if true_items else 0` pour éviter la division par zéro)

7. **Moyenne globale**  
   On agrège les métriques sur tous les utilisateurs et on affiche les résultats finaux pour évaluer la qualité du modèle SVD.



In [ ]:
for n in [10, 20, 50, 100]:
    svd = TruncatedSVD(n_components=n, random_state=42)
    U = svd.fit_transform(R_train)
    Vt = svd.components_
    print(f"\n-- Factors: {n} --")
    evaluate_svd(U, Vt, uid2idx, item_ids, df_test, k=10)


#### Bonus

A partir de cette boucle, on essaye de rouver le meilleur paramètre de composants pour notre SVD en se basant sur la précision et le rappel.

## Analyse des résultats de recommandation

- **SVD (TruncatedSVD)**  
  - *Precision@10* = 0.1590  
  - *Recall@10*    = 0.0077  
  
  La précision faible (≈16 %) indique que, parmi les 10 items recommandés à chaque utilisateur, seuls 1 à 2 sont en moyenne effectivement pertinents. Le rappel quasi nul (<1 %) révèle que le modèle SVD ne couvre quasiment aucune des vidéos réellement regardées par l’utilisateur dans le jeu de test. Cela suggère que les vecteurs latents extraits par SVD sont trop généraux ou mal adaptés aux préférences fines des utilisateurs (sous‑apprentissage ou mauvaise captation des signaux faibles).

- **ALS (Alternating Least Squares)**  
  - *Precision@10* = 0.6715  
  - *Recall@10*    = 0.0272  
  
  Avec près de 67 % de précision, l’ALS propose majoritairement des recommandations pertinentes. Cependant, son rappel reste faible (~2.7 %), ce qui signifie que, bien qu’il “touche juste” quand il recommande, il ne couvre toujours qu’une très petite fraction des vidéos réellement vues.  

### Ce qu'on peut en déduire

1. **Couverture vs. exactitude**  
   - ALS privilégie la “sécurité” : il recommande moins d’items différents (faible rappel) mais quasiment toujours pertinents (haute précision).  
   - SVD, en revanche, disperse ses recommandations dans tout l’espace latent, capturant un peu plus de diversité mais perd en exactitude et passe à côté de presque tous les items pertinents.

2. **Influence du format des données**  
   - ALS est spécifiquement conçu pour les matrices creuses d’implicite (watch_ratio), avec régularisation adaptative, et se montre mieux calibré sur des interactions binaires/pondérées.  
   - SVD linéaire « brut » sur ces mêmes données peut souffrir d’un manque de centrage ou de traitement du biais utilisateur/item.

3. **Axes d’amélioration**  
   - **Pour SVD** :  
     - Centrage (soustraction de la moyenne par user/item) avant factorisation.  
     - Ajustement de `n_components` (réduction ou augmentation des facteurs).  
     - Ajout de pondération ou de normalisation (TF-IDF, BM25) sur les interactions.  
   - **Pour ALS** :  
     - Exploration de la régularisation et du nombre d’itérations.  
     - Validation croisée pour optimiser `factors`, `regularization`.  
     - Hybridation avec des features utilisateur/item (métadonnées, tags, catégories) pour augmenter le rappel.

> **Conclusion** : l’ALS l’emporte nettement en précision, mais tous deux montrent un rappel très bas, typique des systèmes de recommandation sur données très creuses. Pour monter en performance globale, il faudra combiner ces approches avec des signaux non implicites et du filtrage hybrides.
